## tokenize function

In [18]:
import os
from glob import glob
import json
import re
import argparse

from transformers import BertTokenizer

# 문장 하나에 대해 processing하는 함수 클래스
class EntityPosMarker:
    """ 한국어 문장 하나에 대해 entity의 위치를 special token으로 나타낸 후,
        tokenize하여 BERT에 들어갈 수 있는 input ids 형태로 변환.
    
    Attributes:
        tokenizer: BertTokenizer(bert-base-multilingual-cased model을 사용).
        args: Args from command line.
    """
    def __init__(self, args=None):
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
        self.err = 0
        self.args = args
    
    def tokenize(self, sentence, subj_pos_range, obj_pos_range):
        """ Entity Marker를 special token 형태로 추가한 후 BERT-input ids로 변환하는 함수.

        Args:
            sentence: 한국어 문장 (type: str)
            subject_pos_range: subject entity의 위치 인덱스 범위 (인덱스는 문자열 인덱스)
            object_pos_range: object entity의 위치 인덱스 범위 (인덱스는 문자열 인덱스)

        Returns:
            tokenized_input_ids: BERT-input ids
            subj_marker_start: subject entity marker 시작 위치 인덱스
            subj_marker_end: subject entity marker 끝 위치 인덱스
            obj_marker_start: object entity marker 시작 위치 인덱스
            obj_marker_end: object entity marker 끝 위치 인덱스

        Example:
            sentence: "한국은 동아시아의 한반도에 위치하고 있다."
            subj_pos_range: [0, 2] ("한국")
            obj_pos_range: [4, 8] ("동아시아")

            1. tokenizer.tokenize(sentence)
                tokens = ['한국', '##은', '동', '##아', '##시아', '##의', '한', '##반', '##도에', '위', '##치', '##하고', '있다', '.']
            2. special token 추가
                tokenized_sentence = ['[CLS]', '[unused1]', '한국', '[unused2]', '##은', '[unused3]', '동', '##아', '##시아', 
                                    '[unused4]', '##의', '한', '##반', '##도에', '위', '##치', '##하고', '있다', '.', '[SEP]']
            3. tokenized sentence를 BERT-input ids로 변환 및 entities 위치 찾아 return
                tokenized_input_ids = [101, 1, 48556, 2, 10892, 3, 9095, 16985, 46861, 4, 10459, 9954, 30134, 108521,
                                    9619, 18622, 12453, 11506, 119, 102]
        """
        # subj_name, obj_name
        subj_name = sentence[subj_pos_range[0]:subj_pos_range[1]]
        obj_name = sentence[obj_pos_range[0]:obj_pos_range[1]]

        subj_start_idx, subj_end_idx = [], []
        obj_start_idx, obj_end_idx = [], []

        # 1. tokenizer.tokenize(sentence)
        tokens = self.tokenizer.tokenize(sentence)

        # subj, obj token 위치 찾기
        for i, token in enumerate(tokens):
            if token[0] == subj_name[0]:
                subj_start_idx.append(i)
            if token[-1] == subj_name[-1]:
                subj_end_idx.append(i)
        
            if token[0] == obj_name[0]:
                obj_start_idx.append(i)
            if token[-1] == obj_name[-1]:
                obj_end_idx.append(i)
        
        # subj token idx
        subj_flag = False
        for i in subj_start_idx:
            for j in subj_end_idx:
                # tmp_tokens = ['한국']
                tmp_tokens = tokens[i:j+1]
                subj_cand = " ".join(tmp_tokens)
                subj_cand = re.sub(" ##", "", subj_cand)
                if subj_cand == subj_name:
                    subj_token_start, subj_token_end = i, j+1
                    subj_flag = True
                    break
                    
                ##########
                else:
                    print(token)
                    print(subj_cand, subj_name)
            if subj_flag:
                break

        # obj token idx
        obj_flag = False
        for i in obj_start_idx:
            for j in obj_end_idx:
                # tmp_tokens = ['동', '##아', '##시아']
                tmp_tokens = tokens[i:j+1]
                obj_cand = " ".join(tmp_tokens)
                # obj_cand = '동 ##아 ##시아'
                obj_cand = re.sub(" ##", "", obj_cand)
                # obj_cand = '동아시아'
                if obj_cand == obj_name:
                    obj_token_start, obj_token_end = i, j+1
                    obj_flag = True
                    break
                ############3
                else:
                    print(token)
                    print(obj_cand, obj_name)
            if obj_flag:
                break

        # subj_tokens = tokens[subj_token_start:subj_token_end]
        # obj_tokens = tokens[obj_token_start:obj_token_end]


        # 2. special token 추가
        tokenized_sentence = []
        for i, token in enumerate(tokens):
            if i == subj_token_start:
                tokenized_sentence.append("[unused1]")
            elif i == obj_token_start:
                tokenized_sentence.append("[unused3]")

            if i == subj_token_end:
                tokenized_sentence.append("[unused2]")
            elif i == obj_token_end:
                tokenized_sentence.append("[unused4]")

            tokenized_sentence.append(token)

        tokenized_sentence = ["[CLS]"] + tokenized_sentence + ["[SEP]"]

        # 3. entity marker token 위치 찾기
        try:
            subj_marker_start = tokenized_sentence.index("[unused1]")
            subj_marker_end = tokenized_sentence.index("[unused2]")
        except:
            self.err += 1
            subj_marker_start = 0
            subj_marker_end = 2
        
        try:
            obj_marker_start = tokenized_sentence.index("[unused3]")
            obj_marker_end = tokenized_sentence.index("[unused4]")
        except:
            self.err += 1
            obj_marker_start = 0
            obj_marker_end = 2

        tokenized_input_ids = self.tokenizer.convert_tokens_to_ids(tokenized_sentence)

        return tokenized_input_ids, subj_marker_start, subj_marker_end, obj_marker_start, obj_marker_end
        

## train data 확인

In [2]:
train_data = []
with open("../data/train.txt") as f:
    lines = f.readlines()
    for line in lines:
        item = json.loads(line)
        train_data.append(item)

In [3]:
print(f"Train data의 개수: {len(train_data)}")

Train data의 개수: 2685657


In [4]:
train_data[0]

{'file_name': '/data/modified_KBN_data/wikipedia_0001.json',
 'sentence': '제임스 얼 "지미"카터 주니어(, 1924년 10월 1일 ~ )는 민주당 출신 미국 39번째 대통령 (1977년 ~ 1981년)이다.',
 'subj': {'name': '제임스 얼', 'pos': [0, 5], 'type': 'PS'},
 'obj': {'name': '카터 주니어', 'pos': [10, 16], 'type': 'PS'},
 'description': '항목 주제의 후임자',
 'confidence': 0.8254553079605103}

In [5]:
train_data[1]

{'file_name': '/data/modified_KBN_data/wikipedia_0001.json',
 'sentence': '제임스 얼 "지미"카터 주니어(, 1924년 10월 1일 ~ )는 민주당 출신 미국 39번째 대통령 (1977년 ~ 1981년)이다.',
 'subj': {'name': '제임스 얼', 'pos': [0, 5], 'type': 'PS'},
 'obj': {'name': '미국', 'pos': [44, 46], 'type': 'LC'},
 'description': '항목 주제의 국가',
 'confidence': 0.5040757060050964}

## EntityPosMarker 사용 예시

In [19]:
marker = EntityPosMarker()

example = train_data[0]

sentence = example['sentence']
subj_pos_range = example['subj']['pos']
obj_pos_range = example['obj']['pos']

In [ ]:
for i, item in enumerate(train_data):
    sentence, subj_pos_range, obj_pos_range = item['sentence'], item['subj']['pos'], item['obj']['pos']
    try:
        marker.tokenize(sentence, subj_pos_range, obj_pos_range)
    except:
        print(sentence, subj_pos_range, obj_pos_range)

.
이집트와 이스라엘 이스라엘
.
 안와르 사다트
카터는 이집트와 이스라엘을 조정하여, 캠프 데이비드에서 안와르 사다트 대통령과 메나헴 베긴 수상과 함께 중동 평화를 위한 캠프데이비드 협정을 체결했다. [0, 2] [44, 50]
.
 안와르 사다트
.
 안와르 사다트
.
 안와르 사다트
.
이집트와 이스라엘 이스라엘
.
 안와르 사다트
카터는 이집트와 이스라엘을 조정하여, 캠프 데이비드에서 안와르 사다트 대통령과 메나헴 베긴 수상과 함께 중동 평화를 위한 캠프데이비드 협정을 체결했다. [31, 38] [44, 50]
카터는 이집트와 이스라엘을 조정하여, 캠프 데이비드에서 안와르 사다트 대통령과 메나헴 베긴 수상과 함께 중동 평화를 위한 캠프데이비드 협정을 체결했다. [44, 50] [0, 2]
카터는 이집트와 이스라엘을 조정하여, 캠프 데이비드에서 안와르 사다트 대통령과 메나헴 베긴 수상과 함께 중동 평화를 위한 캠프데이비드 협정을 체결했다. [44, 50] [4, 7]
.
이집트와 이스라엘 이스라엘
카터는 이집트와 이스라엘을 조정하여, 캠프 데이비드에서 안와르 사다트 대통령과 메나헴 베긴 수상과 함께 중동 평화를 위한 캠프데이비드 협정을 체결했다. [44, 50] [9, 13]
그러나 이것은 공화당과 미국의 유대인 단체의 반발을 일으켰다. [8, 11] [13, 15]
.
 한
인권 문제와 주한미군 철수 문제로 한때 한미 관계가 불편하기도 했다. [7, 11] [23, 24]
.
 북한
1978년 대한민국에 대한 북한의 위협에 대비해 한미연합사를 창설하면서, 1982년까지 3단계에 걸쳐 주한미군을 철수하기로 했다. [27, 32] [15, 17]
1978년 대한민국에 대한 북한의 위협에 대비해 한미연합사를 창설하면서, 1982년까지 3단계에 걸쳐 주한미군을 철수하기로 했다. [57, 61] [6, 10]
.
 북한
1978년 대한민국에 대한 북한의 위협에 대비해 한미연합사를 창설하면서, 1982년까지 3단계에 걸쳐 주한미군을 철수하기로 했다. [57, 61

 유석동
.
宋 斗 用 宋斗用
동경고등사범학교 재학 중에 일본인 무교회주의자 우치무라(內村鑑三)의 성서연구에 깊이 영향을 받고 김교신(金敎臣), 송두용(宋斗用), 정상훈(鄭相勳), 유석동(柳錫東), 양인성(楊仁性)등과 함께 교회에 다니지 않고도 신앙을 유지하는 무교회주의 신앙클럽을 결성하였다. [84, 87] [68, 71]
.
 유석동
.
 유석동
.
鄭 相 勳 鄭相勳
동경고등사범학교 재학 중에 일본인 무교회주의자 우치무라(內村鑑三)의 성서연구에 깊이 영향을 받고 김교신(金敎臣), 송두용(宋斗用), 정상훈(鄭相勳), 유석동(柳錫東), 양인성(楊仁性)등과 함께 교회에 다니지 않고도 신앙을 유지하는 무교회주의 신앙클럽을 결성하였다. [84, 87] [78, 81]
.
 유석동
.
柳 錫 東 柳錫東
동경고등사범학교 재학 중에 일본인 무교회주의자 우치무라(內村鑑三)의 성서연구에 깊이 영향을 받고 김교신(金敎臣), 송두용(宋斗用), 정상훈(鄭相勳), 유석동(柳錫東), 양인성(楊仁性)등과 함께 교회에 다니지 않고도 신앙을 유지하는 무교회주의 신앙클럽을 결성하였다. [84, 87] [88, 91]
.
 유석동
.
 양인성
.
 유석동
.
楊 仁 性 楊仁性
동경고등사범학교 재학 중에 일본인 무교회주의자 우치무라(內村鑑三)의 성서연구에 깊이 영향을 받고 김교신(金敎臣), 송두용(宋斗用), 정상훈(鄭相勳), 유석동(柳錫東), 양인성(楊仁性)등과 함께 교회에 다니지 않고도 신앙을 유지하는 무교회주의 신앙클럽을 결성하였다. [84, 87] [98, 101]
.
柳 錫 東 柳錫東
.
金 敎 臣 金敎臣
동경고등사범학교 재학 중에 일본인 무교회주의자 우치무라(內村鑑三)의 성서연구에 깊이 영향을 받고 김교신(金敎臣), 송두용(宋斗用), 정상훈(鄭相勳), 유석동(柳錫東), 양인성(楊仁性)등과 함께 교회에 다니지 않고도 신앙을 유지하는 무교회주의 신앙클럽을 결성하였다. [88, 91] [58, 61]
.
柳 錫 東 柳錫東
동경고등사범학교 재학 중에 일본인 무교회주의자 우치무라(內村鑑三)의 

.
나츠메 소세키 , 아쿠타가와 류노스케 , 최근의 작가로는 나카지마 아츠시 나카지마 아츠시
.
나츠메 소세키 , 아쿠타가와 류노스케 , 최근의 작가로는 나카지마 아츠시 , 다자이 오사무 , 하야시 나카지마 아츠시
.
나츠메 소세키 , 아쿠타가와 류노스케 , 최근의 작가로는 나카지마 아츠시 , 다자이 오사무 , 하야시 후미코 , 미야모토 유리코 , 호리 다쓰오 , 사카구치 안고 , 다카무라 고타로 , 나가이 가후 , 요시 나카지마 아츠시
.
 미야모토 유리코
.
 호리 다쓰오
.
 호리 다쓰오
.
 사카구치 안고
.
다자이 오사무 , 하야시 후미코 , 미야모토 유리코 , 호리 다쓰오 , 사카구치 안고 , 다카무라 고타로 다카무라 고타로
.
다쓰오 , 사카구치 안고 , 다카무라 고타로 다카무라 고타로
.
나츠메 소세키 , 아쿠타가와 류노스케 , 최근의 작가로는 나카지마 아츠시 , 다자이 오사무 , 하야시 후 나가이 가후
.
나츠메 소세키 , 아쿠타가와 류노스케 , 최근의 작가로는 나카지마 아츠시 , 다자이 오사무 , 하야시 후미코 , 미야모토 유리코 , 호리 다쓰오 , 사카구치 안고 , 다카무라 고타로 , 나가이 가후 나가이 가후
.
나카지마 아츠시 , 다자이 오사무 , 하야시 후 나가이 가후
.
나카지마 아츠시 , 다자이 오사무 , 하야시 후미코 , 미야모토 유리코 , 호리 다쓰오 , 사카구치 안고 , 다카무라 고타로 , 나가이 가후 나가이 가후
.
 나가이 가후
.
 요시카와 에이지
.
나츠메 소세키 , 아쿠타가와 류노스케 , 최근의 작가로는 나카지마 아츠시 나카지마 아츠시
.
나츠메 소세키 , 아쿠타가와 류노스케 , 최근의 작가로는 나카지마 아츠시 , 다자이 오사무 , 하야시 나카지마 아츠시
.
나츠메 소세키 , 아쿠타가와 류노스케 , 최근의 작가로는 나카지마 아츠시 , 다자이 오사무 , 하야시 후미코 , 미야모토 유리코 , 호리 다쓰오 , 사카구치 안고 , 다카무라 고타로 , 나가이 가후 , 요시 나카지마 아츠시
.
 미야모토 유리코
.
 사카구치 안고


.
아인 아인슈타인
.
아인 아인슈타인
.
다비트 다비트 힐베르트
.
아인 아인슈타인
.
 힐베르트
.
아인 아인슈타인
데니스 매캘리스테어 리치(, 1941년 9월 9일 ~ 2011년 10월 12일)는 미국의 저명한 전산학자이자 현대 컴퓨터의 선구자이다. [0, 13] [46, 48]
C와 유닉스의 개발자로 알려져있다. [3, 6] [0, 1]
미국의 뉴욕 주 브롱스빌(Bronxville)에서 태어났으며, 1967년 하버드 대학교에서 물리학과 응용수학 학위를 얻었다. [0, 2] [14, 24]
미국의 뉴욕 주 브롱스빌(Bronxville)에서 태어났으며, 1967년 하버드 대학교에서 물리학과 응용수학 학위를 얻었다. [41, 48] [0, 2]
미국의 경제 전문지 '비즈니스 인사이더'에서는 '현재의 애플 컴퓨터는 거의 모두 데니스 리치의 업적에 기반하고 있다'이라며 그의 업적을 평가했다. [31, 33] [0, 2]
멘델레예프의 문제는 영국의 모즐리에 의해 풀렸다. [15, 18] [0, 5]
.
変 体 仮 名 変体仮名
위 이외의 구식 자형은 헨타이가나(変体仮名)라고 부른다. [13, 18] [19, 23]
.
 Golo Mann
.
 Golo Mann
.
 율리아 다 실바 브룬스
.
 율리아 다 실바 브룬스
.
토마스 만 토마스 요한 하인리히 만
.
토마스 만은 평의원이며 곡물 상인이었던 토마스 요한 하인리히 만 토마스 요한 하인리히 만
.
 토마스 요한 하인리히 만
.
 율리아 다 실바 브룬스
.
 율리아 다 실바 브룬스
.
 율리아 다 실바 브룬스
.
 율리아 다 실바 브룬스
.
 율리아 다 실바 브룬스
.
 율리아 다 실바 브룬스
.
토마스 만 토마스 요한 하인리히 만
.
토마스 만은 평의원이며 곡물 상인이었던 토마스 요한 하인리히 만 토마스 요한 하인리히 만
.
 토마스 요한 하인리히 만
.
기술 분야에 참가하면서 , 뮌헨 대학 기술대학
.
기술 분야에 참가하면서 , 뮌헨 대학과 기술대학 기술대학
.
 기술대학
.
리투아 리투아니아
.
 스위스
.


김근태 , 김중권 김중권
.
 이인제
.
김근태 , 김중권 김중권
.
김근태 , 김중권 김중권
.
김근태 , 김중권 김중권
.
김근태 , 김중권 김중권
.
 유종근
.
김근태 , 김중권 김중권
.
 이인제
.
김근태 , 김중권 김중권
.
 유종근
.
 유종근
.
 이인제
.
 이인제
.
 이인제
.
김근태 , 김중권 김중권
.
 이인제
.
김근태 , 김중권 김중권
.
 이인제
.
이전에 민주당 부동의 1위는 이인제 이인제
.
이전에 민주당 부동의 1위는 이인제 이인제
.
광주는 김대중 대통령의 정치적 기반이자 새천년민주 광주
.
광주는 김대중 대통령의 정치적 기반이자 새천년민주당의 근거지로서 이곳의 결과가 사실상 새천년민주 광주
무엇보다도 광주는 김대중 대통령의 정치적 기반이자 새천년민주당의 근거지로서 이곳의 결과가 사실상 새천년민주당 대선후보를 결정짓는다고 해도 과언이 아닐 정도로 최대의 승부처였다. [10, 13] [6, 8]
.
 영남
.
광주시민 여러분들의 위대한 승리 , 민주 광주
.
광주시민 여러분들의 위대한 승리 , 민주당의 승리 , 한국 민주 광주
.
광주시민 여러분들의 위대한 승리 , 민주당의 승리 , 한국 민주주의 승리로 이어질 수 있도록 최선을 다하겠습니다 " 라면서 감격적인 소감을 밝혔고 , 이후 노무현은 단숨에 지지율이 급상승하며 ' 노풍 ( 盧 風 , 노무현 바람 ) ' 의 주 광주
노무현은 당시 연단에 서서 "광주시민 여러분들의 위대한 승리, 민주당의 승리, 한국 민주주의 승리로 이어질 수 있도록 최선을 다하겠습니다"라면서 감격적인 소감을 밝혔고, 이후 노무현은 단숨에 지지율이 급상승하며 '노풍(盧風, 노무현 바람)'의 주인공이 되었다. [0, 3] [16, 18]
.
광주시민 여러분들의 위대한 승리 , 민주 광주
.
광주시민 여러분들의 위대한 승리 , 민주당의 승리 , 한국 민주 광주
.
광주시민 여러분들의 위대한 승리 , 민주당의 승리 , 한국 민주주의 승리로 이어질 수 있도록 최선을 다하겠습니다 " 라면서 감격적인 소감을 밝혔고

...

 동아시아
.
동아 동아시아
한국(韓國), 조선(朝鮮), 또는 코리아(Korea)는 동아시아에 위치한 지역 또는 헌법상의 국가로, 현대사에서는 한반도의 대한민국과 조선민주주의인민공화국을 이르는 말이다. [23, 28] [64, 67]
.
 동아시아
.
동아 동아시아
한국(韓國), 조선(朝鮮), 또는 코리아(Korea)는 동아시아에 위치한 지역 또는 헌법상의 국가로, 현대사에서는 한반도의 대한민국과 조선민주주의인민공화국을 이르는 말이다. [31, 35] [64, 67]
.
 동아시아
.
동아 동아시아
한국(韓國), 조선(朝鮮), 또는 코리아(Korea)는 동아시아에 위치한 지역 또는 헌법상의 국가로, 현대사에서는 한반도의 대한민국과 조선민주주의인민공화국을 이르는 말이다. [64, 67] [31, 35]
.
 대한민국
.
 동아시아
.
동아 동아시아
.
 대한민국
한국(韓國), 조선(朝鮮), 또는 코리아(Korea)는 동아시아에 위치한 지역 또는 헌법상의 국가로, 현대사에서는 한반도의 대한민국과 조선민주주의인민공화국을 이르는 말이다. [69, 73] [64, 67]
.
 조선민주주의인민공화국
.
조선 ( 朝 鮮 ) , 또는 코리아 ( Korea ) 는 동아시아에 위치한 지역 또는 헌법상의 국가로 , 현대사에서는 한반도의 대한민국 조선민주주의인민공화국
.
조선 ( 朝 鮮 ) , 또는 코리아 ( Korea ) 는 동아시아에 위치한 지역 또는 헌법상의 국가로 , 현대사에서는 한반도의 대한민국과 조선민주주의인민공화국 조선민주주의인민공화국
.
 조선민주주의인민공화국
.
 조선민주주의인민공화국
.
 동아시아
.
동아 동아시아
.
 조선민주주의인민공화국
.
조선 ( 朝 鮮 ) , 또는 코리아 ( Korea ) 는 동아시아에 위치한 지역 또는 헌법상의 국가로 , 현대사에서는 한반도의 대한민국 조선민주주의인민공화국
.
조선 ( 朝 鮮 ) , 또는 코리아 ( Korea ) 는 동아시아에 위치한 지역 또는 헌법상의 국가로 , 현대사에서는 한반도의 대한민국과 조선민주주의인민공화국 조선

 동남아시아
.
동남아 동남아시아
.
 중앙아시아
.
중앙아 중앙아시아
.
 동남아시아
.
 동남아시아
.
 동남아시아
.
 동남아시아
.
 동남아시아
.
동남아 동남아시아
.
 남아시아
.
 남아시아
.
 남아시아
.
남아 남아시아
.
 동남아시아
.
 동남아시아
.
 동남아시아
.
 동남아시아
.
 동남아시아
.
동남아 동남아시아
.
 중국
.
중앙아시아의 북방계와 남아시아 , 동남아시아의 남방계가 혼합된 바탕에 중국 중국
.
 중국
.
 동남아시아
.
 동남아시아
.
 동남아시아
.
 동남아시아
.
 동남아시아
.
동남아 동남아시아
.
 동남아시아
.
 동남아시아
.
 동남아시아
.
 동남아시아
.
 동남아시아
.
동남아 동남아시아
.
 6광역시
.
이 같은 행정구역 구분을 인정하지 않고 , 광복 당시의 행정구역만을 인정한다 ( 이북5도위원회 이북5도위원회
.
아르키메데스의 계산이 널리 알려져 있어 아르키메데스 아르키메데스
원주율을 나타내는 기호 π는 1706년 영국의 수학자 윌리엄 존스가 최초로 사용했다. [30, 36] [22, 24]
컴퓨터를 도입하기 이전에 가장 긴 자리수의 원주율을 계산한 사람은 영국의 수학자 샹크스였다. [45, 48] [37, 39]
.
近 藤 茂 近藤茂
2010년 8월 3일에는 일본의 회사원 곤도 시게루(近藤茂)가 소수점 이하 5조 자리까지 계산하였다. [22, 28] [29, 32]
.
近 藤 茂 近藤茂
2010년 8월 3일에는 일본의 회사원 곤도 시게루(近藤茂)가 소수점 이하 5조 자리까지 계산하였다. [29, 32] [14, 16]
.
프랑스 수학자 프랑수아 비에트 프랑수아 비에트
.
 요한 베르누이
.
브뤼셀의 브뤼셀 자유대학 브뤼셀 자유대학
.
브뤼셀의 브뤼셀 자유대학 ( Université Libre de Bruxelles ) 에서 화학 브뤼셀 자유대학
.
브뤼셀의 브뤼셀 자유대학 브뤼셀 자유대학
.
브뤼셀의 브뤼셀 자유대학 ( Université Libre de Bruxelles ) 에서 화학 브뤼셀 자유대학
즉

프랑스의 미니텔 서비스를 모델로 한 것으로 , 미니텔과 마찬가지로 문자 및 비디오텍스 프랑스
이러한 형태는 프랑스의 미니텔 서비스를 모델로 한 것으로, 미니텔과 마찬가지로 문자 및 비디오텍스(VTX)가 지원되는 전용 단말기 하이텔 단말기를 도입하여 전화가입자들에게 무상임대하였다. [73, 76] [8, 11]
그 이후, 리니지 등의 게임이나, 디시인사이드같은 유저포탈의 생성으로 즐, 원츄, 아햏햏, 뷁 등의 새로운 단어가 생겨났다. [6, 9] [51, 52]
그 이후, 리니지 등의 게임이나, 디시인사이드같은 유저포탈의 생성으로 즐, 원츄, 아햏햏, 뷁 등의 새로운 단어가 생겨났다. [42, 44] [51, 52]
그 이후, 리니지 등의 게임이나, 디시인사이드같은 유저포탈의 생성으로 즐, 원츄, 아햏햏, 뷁 등의 새로운 단어가 생겨났다. [51, 52] [39, 40]
그 이후, 리니지 등의 게임이나, 디시인사이드같은 유저포탈의 생성으로 즐, 원츄, 아햏햏, 뷁 등의 새로운 단어가 생겨났다. [51, 52] [42, 44]
클린턴 리처드 도킨스(, 1941년 3월 26일~ )는 영국의 동물행동학자, 진화생물학자 및 대중과학 저술가이다. [4, 11] [31, 33]
그는 영국왕실이 수여하는 패러데이 상의 심사위원, 영국TV 아카데미상 심사위원을 맡고 있으며 영국과학발전협회의 생물학 부문 수장이기도 하다. [5, 7] [3, 5]
그는 영국왕실이 수여하는 패러데이 상의 심사위원, 영국TV 아카데미상 심사위원을 맡고 있으며 영국과학발전협회의 생물학 부문 수장이기도 하다. [52, 60] [3, 5]
도킨스는 무신론자 이며, 철저한 인본주의자, 회의주의자, 과학적 합리주의자 및 브라이트 운동 지지자이다. [44, 51] [0, 3]
.
 로트바일러
그는 미디어에서 여러 차례 "다윈의 로트바일러"로 불렸는데, 이는 영국의 생물학자 토머스 헉슬리가 자연 선택을 지지하면서 "다윈의 불독"으로 불린 것에서 유추되었다. [46, 53] [16, 18]
.
 로트바일러
그는 

대한민국 내에서도 직접 이 허가서가 합법한가에 대한 법원의 판단은 없었으며, 다만 허가서라는 특성상 준법행위를 하는 다수는 분쟁없이 이 허가서의 제약을 따르고 있을 뿐이다. [29, 31] [0, 4]
대한민국 내에서도 한 차례 법원의 판단을 받을 뻔했던 엘림넷과 하이온넷 사건이 있었으나, 회사 기밀 유출 사건으로 형사기소되는 바람에 중요 쟁점일 수 있었던 GPL에 대한 판단은 형사법원에서 다루지 않고 다른 이유로 1, 2심 법원에서 유죄를 판결, 최종 확정한 바 있다. [100, 104] [0, 4]
대한민국 내에서도 한 차례 법원의 판단을 받을 뻔했던 엘림넷과 하이온넷 사건이 있었으나, 회사 기밀 유출 사건으로 형사기소되는 바람에 중요 쟁점일 수 있었던 GPL에 대한 판단은 형사법원에서 다루지 않고 다른 이유로 1, 2심 법원에서 유죄를 판결, 최종 확정한 바 있다. [127, 129] [0, 4]
GPL은 미국의 리처드 스톨만(Richard Stallman)이 GNU-프로젝트로 배포된 프로그램의 라이선스로 사용하기 위하여 작성하였다. [5, 7] [0, 3]
.
GNU 디버거 ( GDB GDB
.
GNU 디버거 ( GDB ) , GNU 컴파일러 모음 ( GC GCC
.
GNU 디버거 ( GDB ) , GNU 컴파일러 모음 ( GCC GCC
.
GDB ) , GNU 컴파일러 모음 ( GC GCC
.
GDB ) , GNU 컴파일러 모음 ( GCC GCC
.
GNU 컴파일러 모음 ( GC GCC
.
GNU 컴파일러 모음 ( GCC GCC
.
GC GCC
.
 가타카나
.
 가타카나
.
 하와이
.
 하와이
.
 북마리아나 제도
.
 북마리아나 제도
.
 북마리아나 제도
.
 북마리아나 제도
.
 북마리아나 제도
.
 북마리아나 제도
.
일본어를 공용어의 하나로 채용하고 있지만 현재 앙가우르 주 내에는 일본어를 일상 회화에 쓰는 주민은 존재하지 않아 실질적인 주 공용어의 역할을 하고 있지 않으며 , 일본국 일본국
.
일본어를 일상 회화에 쓰는 주민은 존재하지 않

.
 정부당국
.
 정부당국
.
 정부당국
.
정당 당사 등을 100여차례 기습 또는 항의 방문했으며 화염병과 쇠파이프 , 돌 등을 동원한 시위로 중상자 36명을 포함한 경찰관 413명이 부상을 입고 경찰차량 10대를 파손시키기도 하여 정부당국 정부당국
.
 정부당국
.
 정부당국
.
 정부당국
.
 북한
.
 북한
.
 북한
.
 북한
.
 중국
.
金 在 容 한양대총학생회장 ) 은 1993년 5월 29일 오전8시10분부터 고려대 학생회관 1층 생활도서관에서 제1기 출범식 행사의 하나로 마련 金在容한총련
.
金 在 容 한양대총학생회장 ) 은 1993년 5월 29일 오전8시10분부터 고려대 학생회관 1층 생활도서관에서 제1기 출범식 행사의 하나로 마련한 북한및 해외 학생대표들과 국제전화를 통해 「 조국통일 범민족청년학생연합 」 金 在 容 한총련 金在容한총련
.
 金在容한총련
.
金 在 容 한총련 金在容한총련
.
 북한
.
 북한
의장 金在容한양대총학생회장)은 1993년 5월 29일 오전8시10분부터 고려대 학생회관 1층 생활도서관에서 제1기 출범식 행사의 하나로 마련한 북한및 해외 학생대표들과 국제전화를 통해 「조국통일 범민족청년학생연합」金在容한총련의장 등 남측본부 의장단 11명은 서울에서, 허창조 조선학생위원장 등 6명의 북측 본부 의장단은 중국 북경 연경호텔에서, 김창오 재일한국청년동맹위원장 등 해외본부 의장단 6명은 일본 동경에서 국제전화로 회의를 열고 스피커를 통해 공개적으로 2시간여 동안 통일방안과 제3차 청년학생통일축전 등에 대해 논의했다. [119, 125] [80, 82]
.
 허창조
.
 허창조
.
북한및 해외 학생대표들과 국제전화를 통해 「 조국통일 범민족청년학생연합 」 金 在 容 한총련의장 등 남측본부 의장단 11명은 서울에서 , 허창조 조선학생위원장 등 6명의 북측 본부 의장단은 중국 북경 북경
.
북한및 해외 학생대표들과 국제전화를 통해 「 조국통일 범민족청년학생연합 」 金 在 容 한총련의장 등 남측본부 의장단 11명은 서울에서 , 허창

In [ ]:
print(marker.tokenize(sentence=sentence, subj_pos_range=subj_pos_range, obj_pos_range=obj_pos_range))

In [ ]:
tokenized_input_ids, subj_marker_start, subj_marker_end, obj_marker_start, obj_marker_end = marker.tokenize(sentence, subj_pos_range, obj_pos_range)

In [19]:
print(f"Input ids: \n{tokenized_input_ids}")

Input ids: 
[101, 1, 9672, 36240, 12605, 9551, 2, 107, 9706, 22458, 107, 3, 9786, 21876, 9689, 25503, 12965, 4, 113, 117, 11416, 10954, 16650, 18329, 198, 114, 9043, 9311, 16323, 21928, 9768, 25387, 23545, 11303, 48506, 70672, 113, 10722, 10954, 198, 96318, 114, 30919, 119, 102]


In [17]:
print(f"Subj entity marker 위치: [{subj_marker_start}, {subj_marker_end}]")
print(f"Obj entity marker 위치: [{obj_marker_start}, {obj_marker_end}]")

Subj entity marker 위치: [1, 6]
Obj entity marker 위치: [11, 17]


In [25]:
print(f"Original Sentence: {sentence}")
print(f"Subj name: {sentence[subj_pos_range[0]:subj_pos_range[1]]}")
print(f"Obj name: {sentence[obj_pos_range[0]:obj_pos_range[1]]}")

Original Sentence: 제임스 얼 "지미"카터 주니어(, 1924년 10월 1일 ~ )는 민주당 출신 미국 39번째 대통령 (1977년 ~ 1981년)이다.
Subj name: 제임스 얼
Obj name: 카터 주니어


In [22]:
print(marker.tokenizer.convert_ids_to_tokens(tokenized_input_ids))

['[CLS]', '[unused1]', '제', '##임', '##스', '얼', '[unused2]', '"', '지', '##미', '"', '[unused3]', '카', '##터', '주', '##니', '##어', '[unused4]', '(', ',', '1924', '##년', '10월', '1일', '~', ')', '는', '민', '##주', '##당', '출', '##신', '미국', '39', '##번째', '대통령', '(', '1977', '##년', '~', '1981년', ')', '이다', '.', '[SEP]']


원래 문장이 tokenize된 후 entity marker의 역할을 하는 special token이 추가된 형태로 BERT-input ids로 변환되는 것을 확인할 수 있습니다.